## 测试spliter功能

In [1]:
from dataloaders import Cifar100_Spliter

spliter = Cifar100_Spliter(client_num=5, task_num=5, private_class_num=15, input_size=224, path='C:/Users/Admin/datasets')
client_subset,client_mask = spliter.random_split()

print(client_mask)

del client_subset,client_mask,spliter

c:\Users\Admin\anaconda3\envs\powder\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified


100%|██████████| 50000/50000 [00:01<00:00, 38430.04it/s]


[[[94, 71, 78, 96, 77, 91, 72, 68], [82, 18, 29, 38, 83, 51, 63, 0], [17, 99, 87, 50, 31, 43, 16, 6], [7, 14, 66, 23, 62, 74, 76, 32], [97, 53, 84, 30, 79, 33, 28, 65]], [[96, 66, 15, 12, 51, 38, 67, 92], [29, 98, 30, 55, 19, 84, 33, 74], [52, 62, 77, 76, 91, 18, 99, 85], [97, 10, 2, 5, 13, 63, 34, 79], [7, 65, 0, 11, 17, 50, 31, 78]], [[78, 18, 64, 58, 97, 75, 65, 1], [41, 91, 38, 99, 77, 17, 66, 62], [8, 63, 37, 0, 24, 74, 51, 84], [76, 30, 4, 73, 29, 31, 59, 50], [70, 7, 79, 96, 35, 61, 60, 33]], [[51, 77, 33, 63, 18, 36, 50, 90], [40, 62, 45, 49, 20, 97, 29, 57], [30, 31, 99, 27, 96, 54, 42, 79], [66, 47, 7, 91, 17, 65, 38, 78], [39, 74, 0, 76, 81, 84, 93, 48]], [[0, 3, 89, 76, 63, 96, 51, 84], [30, 91, 86, 33, 50, 65, 9, 66], [31, 74, 25, 62, 26, 7, 99, 21], [17, 44, 88, 69, 77, 22, 79, 56], [78, 80, 18, 29, 38, 95, 46, 97]]]


In [1]:
from dataloaders import ImageNetR_Spliter

spliter = ImageNetR_Spliter(client_num=5, task_num=5, private_class_num=40, input_size=224, path='C:/Users/Admin/datasets')

c:\Users\Admin\anaconda3\envs\powder\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using downloaded and verified file: C:/Users/Admin/datasets\imagenet-r.tar
Using downloaded and verified file: C:/Users/Admin/datasets\imagenet-r.tar


In [2]:
client_subset,client_mask = spliter.random_split()

print(client_mask)

24000it [00:36, 662.42it/s] 

[[[5, 182, 9, 161, 92, 28, 58, 125], [151, 89, 17, 118, 22, 15, 183, 72], [133, 196, 76, 52, 178, 86, 195, 109], [48, 69, 73, 106, 46, 190, 67, 35], [31, 137, 74, 36, 95, 128, 20, 7]], [[144, 33, 96, 143, 43, 141, 168, 51], [59, 135, 83, 170, 156, 107, 4, 91], [55, 39, 167, 30, 65, 153, 13, 79], [99, 129, 0, 62, 34, 80, 85, 2], [45, 10, 3, 23, 131, 16, 159, 19]], [[112, 71, 114, 174, 163, 134, 165, 97], [154, 199, 158, 162, 70, 14, 53, 81], [166, 123, 47, 49, 191, 116, 103, 57], [105, 173, 175, 150, 149, 93, 181, 146], [126, 101, 164, 138, 171, 82, 8, 198]], [[64, 38, 130, 197, 44, 11, 88, 75], [140, 24, 152, 122, 117, 1, 132, 176], [100, 94, 12, 113, 6, 185, 26, 124], [148, 60, 120, 147, 108, 115, 102, 84], [157, 66, 41, 192, 177, 194, 56, 110]], [[121, 77, 29, 160, 32, 90, 37, 111], [98, 187, 169, 179, 189, 40, 87, 145], [127, 27, 136, 172, 180, 119, 18, 42], [139, 21, 193, 68, 142, 63, 50, 186], [54, 61, 155, 25, 78, 104, 188, 184]]]


In [2]:
del client_subset,client_mask,spliter

NameError: name 'client_subset' is not defined